# Testing an 'online' estimator for $R_n$

### Imports

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

### Noise Generator $N_t$
A core assumption of the SPS algorithm is that the noise is zero mean, symmetric and iid

# ARMAX Generator
Class for simulating a system of the form $A(q)y(t) = B(q)u(t) + C(q)e(t) + D$

This class has been developed with the intention of running it in 'online' mode where an update function will be called at each cycle

In [2]:
class ARMAX_Generator:
    def __init__(self, A: list, B: list, C: list, D: list, input_controller) -> None:
        # System Parameters
        self.A = A  # AR coefficients
        self.B = B  # X input coefficients
        self.C = C  # MA coefficients
        self.D = D  # Constant term

        self.order = max(len(A), len(B), len(C)) 

        def temp_input_controller(y, u) -> float: 
            sigma = 0.1
            return 0.4 * y[-1] + np.random.normal(sigma)

        # TODO replace this with the input parameter in the constructor
        self.input_controller = temp_input_controller #input_controller

        # State
        # The below vectors should all be of the same length equal to the current time step
        self.y = []  # Output of the system
        self.u = []  # Input to the system
        self.e = []  # Noise input to the system

    # TODO Replace this with a user-provided noise function which can be any zero-mean symmetric iid noise source
    def generate_noise(self) -> float:
        sigma = 0.1
        self.e.append(np.random.normal(0, sigma))
    
    # Generate new datapoints for the next timestep and append it to the system's state
    def generate_datapoint(self) -> None: 
        # TODO: Replace these with lfilter() calls?
        # AR component
        overlap = min(len(self.A)-1, len(self.y)) # Calculate the overlap between A(2), A(3), A(...) and the past inputs
        ar_sum = np.matmul(self.A[1:overlap+1:][::-1], self.y[-overlap::]) # Reverse the order of the coefficients and multiply the matrices

        # Exogenous input component
        overlap = min(len(self.B), len(self.u))
        exo_sum = np.matmul(self.B[:overlap:][::-1], self.u[-overlap::])

        # MA component
        overlap = min(len(self.C), len(self.e))
        ma_sum = np.matmul(self.C[:overlap:][::-1], self.e[-overlap::])

        # Compute output
        self.y.append( -ar_sum + exo_sum + ma_sum + self.D / self.A[0])
        self.u.append( self.input_controller(self.y, self.u) )

In [3]:
class LqrController:
    def __init__(self) -> None:
        self.is_reuse_noise = True  # Set flag to true if the noise samples should be retained across multiple calculations of output
                                    # This should help with the issue where random noise in the controller output could not be captured when calculating SPS
        self.t = 0
        self.Nt = [] # Vector of noise values

        self.K = 0.5

        self.sigma = 0.001 # Variance of noise generated

    # Generates t length vector of noise values
    def regenerate_noise(self):
        self.Nt = [ np.random.normal(self.sigma) for x in range(self.t)]

    # TODO: Define the logic for the online controller
    def update_gain(self):
        pass

    # This function can be passed into the ARMAX Generator instance
    def generate_output(self, prev_y: float, t: int) -> np.ndarray:
        noise = 0.0

        if self.is_reuse_noise:
            if len(self.Nt) < t+1:
                raise "Error: noise samples not initialised"
            noise = self.Nt[t]
        else:
            noise = np.random.normal(self.sigma)
            if len(self.Nt) < t+1:
                self.Nt = np.concatenate([self.Nt, np.zeros(t - len(self.Nt) + 1)])
            self.Nt[t] = noise
        
        return self.K * prev_y + noise

controller = LqrController()

y = [0 for x in range(50)]
controller.t = len(y)
controller.is_reuse_noise = False
# controller.regenerate_noise()

print(controller.generate_output(y[0], 50))
print(controller.generate_output(y[0], 50))
controller.is_reuse_noise = True
print(controller.generate_output(y[0], 50))


-2.02227635675708
-0.15522499157444108
-0.15522499157444108


# SPS Algorithm

In [15]:
class SPS:
    def __init__(self, p:float=(1-5/40), m:int=40, q:int=5, n:int=0, T:int=1) -> None:
        self.p = p # Confidence probability
        self.m = m # Integer m such that p = 1 − q/m
        self.q = q # Integer q such that p = 1 − q/m
        self.n = n # Number of data points

        self.T = T # block SPS window size
        self.T_repetitions_left: int = 0
        
        self.alpha = None # Random signs
        self.pi = None    # Random permutation

    # Process one time step
    # This is the main function for the real-time SPS implementation
    def update(self) -> None:
        self.n += 1
        self.append_alpha_column()
        self.generate_random_perm()

    # Generate a column of alpha values
    # This should be called whenever we add another data sample (n -> n+1)
    def append_alpha_column(self) -> None:
        new_col = None

        if self.T > 1 and self.T_repetitions_left > 0:
            self.T_repetitions_left -= 1 # Decrement repetitions

            # Repeat the last column in the array
            new_col = self.alpha[:, [-1]]

        else:
            # Check if we need to reset T
            if self.T > 1:
                self.T_repetitions_left = self.T - 1

            # Generate new column of random values
            options = [-1, 1]
            new_col = np.transpose(np.array([[np.random.choice(options) for x in range(self.m)]]))

        # Update the state of the class instance
        if self.alpha is None:
            self.alpha = new_col
        else:
            self.alpha = np.concatenate([self.alpha, new_col], axis=1)

    # Generates m-by-n matrix of integers for ordering SPS indicators
    # TODO Determine if there's anyway to avoid regenerating old values in the pi matrix. 
    def generate_random_perm(self) -> None:
        self.pi = np.array([np.random.permutation(range(self.n)) for x in range(self.m)]) 

    # TODO Test this code
    def least_square_estimator(self, X: np.ndarray, y: np.ndarray) -> np.ndarray:
        theta_hat = np.linalg.lstsq(np.matmul(np.transpose(X), X), np.matmul(np.transpose(X), y))
        
        return theta_hat
    
    def create_feature_matrix(self):
        # This matrix will consist of negative A parameters (except A0) and all B parameters
        # The noise matrix C will be ignored
        pass

    def get_phi(self, A: np.ndarray, B: np.ndarray, C: np.ndarray, y: np.ndarray, u: np.ndarray, t: int) -> np.ndarray:
        num_y_samples = len(A) - 1 # Ignore first A parameter
        num_u_samples = len(B)

        # Calculate how many zeros need to be padded to output
        y_zeros = min(num_y_samples - t, 0)
        u_zeros = min(num_u_samples - t, 0)

        phi_y = y[::-1][1:num_y_samples+1]
        if y_zeros > 0:
            phi_y = np.concatenate([y_zeros * [0], phi_y])

        phi_u = u[::-1][0:num_u_samples]
        if u_zeros > 0:
            phi_u = np.concatenate([u_zeros * [0], phi_u])

        return np.concatenate([phi_y, phi_u])
    
    def sps_indicator(self, y: np.ndarray, u: np.ndarray, A_star: np.ndarray, B_star: np.ndarray, C_star: np.ndarray, D_star: np.ndarray, controller, createFeatureMatrix) -> bool:
        # Since we are using the ARMAX format, we are assuming that the system is closed-loop
        # We must first convert this system into an open loop system so that the SPS method can be used
        # We will be using the 'indirect PEM' method from the 'Closed-Loop Applicability of the Sign-Perturbed Sums Method' paper
        # We are initially identifying the term Kappa which is then converted into the system parameters of interest theta
        
        A_yt = self.basic_filter(A_star, y) # Compute A(q)y_t
        B_ut = self.basic_filter(B_star, u) # Compute B(q)u_t

        diff_AB = A_yt - B_ut

        # Reconstruct the noise C^-1(A(q)y_t - B(q)u_t)
        nt_theta = self.filter([1], C_star, diff_AB)

    def sps_direct(self, y, u, A_star, B_star, C_star, D_star, controller, createFeatureMatrix) -> bool:
        # Reconstruct the noise
        A_yt = self.basic_filter(A_star, y) # Compute A(q)y_t
        B_ut = self.basic_filter(B_star, u) # Compute B(q)u_t

        diff_AB = A_yt - B_ut

        nt_pred = self.filter([1], C_star, diff_AB)

        sp_errors = np.multiply(self.alpha * nt_pred) # Regenerate sign perturbed noise

        # Generate new y_bar(t) = B(q)U_t + alpha_i,t * C(q) * nt_pred(t)

        # TODO: Work out what W and Q vectors are in psi_t(theta) definition
        # Is it just the phi matrix from the other paper? V confusing :) 
        # If so, then phi(t) would be [a1, a2, ... b1, b2, ..., c1, c2, ... ]'
        # This is what we are using the createFeatureMatrix function in matlab for 

    # FIR Filter where the A matrix is [1]
    def basic_filter(self, B: np.ndarray, X: np.ndarray) -> np.ndarray:
        # return np.convolve(B, X)[:len(X)]
        return scipy.signal.lfilter(B, [1.0], X)
    
    def filter(self, B: np.ndarray, A: np.ndarray, X) -> np.ndarray:
        return scipy.signal.lfilter(B, A, X)


# Auxilliary Functions

In [47]:
def create_feature_matrix(y: np.ndarray, u: np.ndarray, nt: np.ndarray):
    if len(y) != len(u):
        raise "Vectors y and u must have the same length"
    
 

# Testing

In [17]:
# SPS Testing
sps = SPS(p=1-1/8, T=1)

for i in range(10):
    sps.update()
    
# print(sps.alpha)
# print(sps.pi)

A = [1, 0.5, 0.25]
B = [1,2,3,4]
C = [1]

y = [1,2,3,4,5]
u = [6,7,8,9,10]
print(sps.basic_filter(B, y))


sps.get_phi(A, B, C, y, u, 2)

[ 1.  4. 10. 20. 30.]


array([ 4,  3, 10,  9,  8,  7])

In [57]:
# Test the ARMAX Generator
theta_real = [0.44, 0.33, 0]
A = [1, -theta_real[0]];    # AR coefficients
B = [0, theta_real[1]];     # MA coefficients
C = [1, theta_real[2]];     # input coefficients
D = 0;                      # Constant term
generator = ARMAX_Generator(A, B, C, D, None)

for i in range(100):
    generator.generate_noise()
    generator.generate_datapoint()



fig = plt.figure()
plt.plot(generator.y)
plt.plot(generator.e)
plt.plot(generator.u)
plt.legend(['Output', 'Noise', 'Input'])
plt.show()

In [127]:
# TODO: PEM Estimate

# Model Orders
na = 1  # Order of A(q)
nb = 1  # Order of B(q)
nc = 0  # Order of C(q)
nk = 1  # Input delay

